In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

from speckit import SpectrumAnalyzer

In [ ]:
# =============================================================================
# 1. ANALYSIS CONFIGURATION
# =============================================================================
# Centralize all parameters for easy tweaking.
ANALYSIS_PARAMS = {
    "olap": 0.75,
    "bmin": 1.0,
    "Lmin": 1000,
    "Jdes": 1000,
    "Kdes": 100,
    "force_target_nf": True,
}

# Define schedulers to test and their plotting styles for consistency.
SCHEDULERS = {
    "lpsd": {"name": "LPSD Scheduler", "color": "deepskyblue", "ls": "-"},
    "ltf": {"name": "LTF Scheduler", "color": "lime", "ls": "--"},
    "new_ltf": {"name": "New LTF Scheduler", "color": "tomato", "ls": "-."},
}

# Consistent plotting aesthetics
FIG_DEFAULTS = {
    "figsize": (7, 5),
    "dpi": 150,
    "fontsize": 9,
    "linewidth": 2.0,
}

In [ ]:
def plot_scheduler_comparison(
    x_data, y_data, title, xlabel, ylabel, xscale="linear", yscale="linear"
):
    """
    Master plotting function to compare scheduler outputs.

    Args:
        x_data (dict): Dictionary of x-axis data, keyed by scheduler name.
        y_data (dict): Dictionary of y-axis data, keyed by scheduler name.
        title (str): Plot title.
        xlabel (str): Label for the x-axis.
        ylabel (str): Label for the y-axis.
        xscale (str): Scale for x-axis ('linear', 'log').
        yscale (str): Scale for y-axis ('linear', 'log').

    Returns:
        tuple: (fig, ax) tuple from matplotlib.
    """
    fig, ax = plt.subplots(figsize=FIG_DEFAULTS["figsize"], dpi=FIG_DEFAULTS["dpi"])

    for key, props in SCHEDULERS.items():
        if key in x_data and key in y_data:
            ax.plot(
                x_data[key],
                y_data[key],
                label=props["name"],
                color=props["color"],
                ls=props["ls"],
                lw=FIG_DEFAULTS["linewidth"],
            )

    ax.set_title(title, fontsize=FIG_DEFAULTS["fontsize"])
    ax.set_xlabel(xlabel, fontsize=FIG_DEFAULTS["fontsize"])
    ax.set_ylabel(ylabel, fontsize=FIG_DEFAULTS["fontsize"])
    ax.tick_params(labelsize=FIG_DEFAULTS["fontsize"])
    ax.set_xscale(xscale)
    ax.set_yscale(yscale)
    ax.grid(True, which="both", linestyle="--", linewidth=0.5)
    ax.legend(
        loc="best",
        edgecolor="black",
        fancybox=True,
        shadow=True,
        framealpha=1,
        fontsize=FIG_DEFAULTS["fontsize"],
    )
    fig.tight_layout()

    return fig, ax

In [ ]:
# Generate random data for the analyzer
N = int(1e6)
fs = 1
random_data = np.random.randn(N)

# =============================================================================
# 2. GENERATE SCHEDULER PLANS
# =============================================================================
print("--- Generating Scheduler Plans ---")
plans = {}
for key, props in SCHEDULERS.items():
    print(f"Running {props['name']}...")
    analyzer = SpectrumAnalyzer(random_data, fs=fs, scheduler=key, **ANALYSIS_PARAMS)
    plans[key] = analyzer.plan()
    # Store the actual Jdes found by the binary search for labeling plots
    plans[key]["final_Jdes"] = analyzer.config["Jdes"]

# =============================================================================
# 3. SANITY CHECKS AND DATA PREPARATION
# =============================================================================
print("\n--- Sanity Checks ---")
# Check that all schedulers produced the same number of frequencies
nfs = [p["nf"] for p in plans.values()]
assert len(set(nfs)) == 1, f"Mismatch in frequency counts: {nfs}"
nf = nfs[0]
print(f"Desired frequencies (Jdes): {ANALYSIS_PARAMS['Jdes']}")
print(f"Actual frequencies produced (nf): {nf}\n")

for key, p in plans.items():
    print(f"Scheduler: {SCHEDULERS[key]['name']}")
    print(f"  Final Jdes used: {p['final_Jdes']}")
    print(
        f"  Frequency range: {p['f'][0]:.4g} Hz to {p['f'][-1]:.4g} Hz (f_max={fs / 2:.4g} Hz)"
    )
    print(
        f"  Segment length range (L): {np.min(p['L'])} to {np.max(p['L'])} (L_min={ANALYSIS_PARAMS['Lmin']})"
    )
    print("-" * 20)

# Prepare common frequency grids for comparison and interpolation
fmin = fs / N
fmax = fs / 2

# Create an ideal logarithmic frequency axis to interpolate onto
f_logspace = np.logspace(np.log10(fmin), np.log10(fmax), ANALYSIS_PARAMS["Jdes"])

# =============================================================================
# 4. INTERPOLATE RESULTS TO A COMMON GRID
# =============================================================================
print("\n--- Interpolating results to common frequency grid ---")
interp_results = {}
quantities_to_interp = ["r", "b", "L", "K", "O"]

for sched_key, plan in plans.items():
    interp_results[sched_key] = {}
    for qty in quantities_to_interp:
        # Create an interpolation function for the current quantity
        interp_func = interp1d(
            plan["f"], plan[qty], kind="linear", bounds_error=False, fill_value=np.nan
        )
        # Evaluate it on the common log-spaced frequency grid
        interp_results[sched_key][qty] = interp_func(f_logspace)

In [ ]:
# =============================================================================
# 5. PLOTTING AND ANALYSIS
# =============================================================================
print("--- Generating Plots ---")

# --- PLOT 1: Frequency Spacing (f vs. index) ---
# This plot shows how each scheduler distributes the frequency points.
# We compare against ideal linear and logarithmic spacing.
x_data = {k: np.arange(p["nf"]) for k, p in plans.items()}
y_data = {k: p["f"] for k, p in plans.items()}
fig, ax = plot_scheduler_comparison(
    x_data,
    y_data,
    title=f"Frequency Point Distribution (N={N:.0e}, Jdes={ANALYSIS_PARAMS['Jdes']})",
    xlabel="Frequency Index, j",
    ylabel="Frequency, f(j) [Hz]",
    yscale="log",
)
# Add ideal reference lines
i_ref = np.arange(1, ANALYSIS_PARAMS["Jdes"] + 1)
ax.plot(
    i_ref,
    np.linspace(fmin, fmax, ANALYSIS_PARAMS["Jdes"]),
    label="Ideal Linspace",
    c="black",
    ls=":",
    lw=1.5,
)
ax.plot(
    i_ref,
    np.logspace(np.log10(fmin), np.log10(fmax), ANALYSIS_PARAMS["Jdes"]),
    label="Ideal Logspace",
    c="gray",
    ls=":",
    lw=1.5,
)
ax.legend(
    loc="best",
    edgecolor="black",
    fancybox=True,
    shadow=True,
    framealpha=1,
    fontsize=FIG_DEFAULTS["fontsize"],
)
plt.show()

# --- PLOT 2: Resolution Bandwidth (r vs. frequency) ---
# This plot visualizes the core strategy of each scheduler. Logarithmic spacing
# implies that resolution 'r' should be proportional to frequency 'f'.
x_data = {k: p["f"] for k, p in plans.items()}
y_data = {k: p["r"] for k, p in plans.items()}
fig, ax = plot_scheduler_comparison(
    x_data,
    y_data,
    title="Resolution Bandwidth vs. Frequency",
    xlabel="Frequency [Hz]",
    ylabel="Resolution Bandwidth, r [Hz]",
    xscale="log",
    yscale="log",
)
# Add constraint lines
fresmin = fs / N
freslim = fresmin * (1 + (1 - ANALYSIS_PARAMS["olap"]) * (ANALYSIS_PARAMS["Kdes"] - 1))
ax.axhline(y=fresmin, color="gray", ls="--", lw=2, label="Min Resolution (fres_min)")
ax.axhline(
    y=freslim, color="purple", ls="--", lw=2, label="Avg Target Resolution (fres_lim)"
)
ax.legend(
    loc="best",
    edgecolor="black",
    fancybox=True,
    shadow=True,
    framealpha=1,
    fontsize=FIG_DEFAULTS["fontsize"],
)
plt.show()

# --- PLOT 3: Bin Number (b vs. frequency) ---
# The fractional bin number b = f/r. For ideal log spacing, this should be constant.
# This plot shows how well each scheduler maintains a constant b.
x_data = {k: p["f"] for k, p in plans.items()}
y_data = {k: p["b"] for k, p in plans.items()}
fig, ax = plot_scheduler_comparison(
    x_data,
    y_data,
    title="Fractional Bin Number vs. Frequency",
    xlabel="Frequency [Hz]",
    ylabel="Fractional Bin Number, b = f/r",
    xscale="log",
    yscale="log",
)
ax.axhline(
    y=ANALYSIS_PARAMS["bmin"], color="red", ls="--", lw=2, label="b_min Constraint"
)
ax.legend(
    loc="best",
    edgecolor="black",
    fancybox=True,
    shadow=True,
    framealpha=1,
    fontsize=FIG_DEFAULTS["fontsize"],
)
plt.show()

# --- PLOT 4a: Comparison of Segment Lengths (L) ---
# This plot directly compares how each scheduler decreases the segment length
# as frequency increases. This is a direct consequence of the resolution 'r' (since L = fs/r).
x_data = {k: p["f"] for k, p in plans.items()}
y_data = {k: p["L"] for k, p in plans.items()}
fig, ax = plot_scheduler_comparison(
    x_data,
    y_data,
    title="Comparison of Segment Length vs. Frequency",
    xlabel="Frequency [Hz]",
    ylabel="Segment Length, L",
    xscale="log",
    yscale="log",
)
ax.axhline(
    y=ANALYSIS_PARAMS["Lmin"], color="red", ls="--", lw=2, label="L_min Constraint"
)
ax.legend(
    loc="best",
    edgecolor="black",
    fancybox=True,
    shadow=True,
    framealpha=1,
    fontsize=FIG_DEFAULTS["fontsize"],
)
plt.show()

# --- PLOT 4b: Comparison of Number of Averages (K) ---
# This plot compares how many segments each scheduler can average at each frequency.
# This is critical for the statistical stability of the spectral estimate.
x_data = {k: p["f"] for k, p in plans.items()}
y_data = {k: p["K"] for k, p in plans.items()}
fig, ax = plot_scheduler_comparison(
    x_data,
    y_data,
    title="Comparison of Number of Averages vs. Frequency",
    xlabel="Frequency [Hz]",
    ylabel="Number of Averages, K",
    xscale="log",
    yscale="log",
)
ax.axhline(
    y=ANALYSIS_PARAMS["Kdes"], color="purple", ls="--", lw=2, label="K_des Target"
)
ax.legend(
    loc="best",
    edgecolor="black",
    fancybox=True,
    shadow=True,
    framealpha=1,
    fontsize=FIG_DEFAULTS["fontsize"],
)
plt.show()

# --- PLOT 5: Overlap (O vs. frequency) ---
# Checks how closely the actual segment overlap matches the desired overlap.
x_data = {k: p["f"] for k, p in plans.items()}
y_data = {k: p["O"] for k, p in plans.items()}
fig, ax = plot_scheduler_comparison(
    x_data,
    y_data,
    title="Actual vs. Desired Overlap",
    xlabel="Frequency [Hz]",
    ylabel="Fractional Overlap",
    xscale="log",
)
ax.axhline(
    y=ANALYSIS_PARAMS["olap"], color="gray", ls="--", lw=2, label="Desired Overlap"
)
ax.legend(
    loc="best",
    edgecolor="black",
    fancybox=True,
    shadow=True,
    framealpha=1,
    fontsize=FIG_DEFAULTS["fontsize"],
)
ax.set_ylim(bottom=ANALYSIS_PARAMS["olap"] * 0.9, top=ANALYSIS_PARAMS["olap"] * 1.1)
plt.show()

In [ ]:
# --- PLOT 6: Constraint Verification (Relative Errors) ---
# This technical plot checks how well two key constraints hold:
# 1. r[j] * L[j] = fs  (DFT constraint)
# 2. r[j] ≈ f[j+1] - f[j] (Frequency stepping constraint)
# Errors are expected to be near machine precision.

fig, ax = plt.subplots(figsize=FIG_DEFAULTS["figsize"], dpi=FIG_DEFAULTS["dpi"])

# --- RESTRUCTURED PLOTTING LOGIC ---

# First, plot the DFT constraint error (r*L = fs) for all schedulers.
# These will use the primary line styles (solid, dashed, etc.).
print("Plotting DFT Constraint Errors (r*L=fs)...")
for key, plan in plans.items():
    props = SCHEDULERS[key]

    # Error in DFT constraint: |r - fs/L| / r
    # This is numerically more stable than calculating |r*L - fs| / (r*L)
    err_dft = np.abs(plan["r"] - (fs / plan["L"])) / plan["r"]

    ax.plot(
        plan["f"],
        err_dft,
        label=f"{props['name']}: r*L=fs",
        color=props["color"],
        ls=props["ls"],  # Use the primary linestyle
        lw=1.5,
    )

# Second, plot the stepping constraint error (r = Δf) for all schedulers.
# We will make all of these dotted to visually group them together.
print("Plotting Stepping Constraint Errors (r=Δf)...")
for key, plan in plans.items():
    props = SCHEDULERS[key]

    # Error in stepping constraint: |r - df| / r
    df = np.diff(plan["f"])
    # Note: The resolution 'r' at index j determines the step to frequency j+1.
    # So we compare r[:-1] with df.
    err_step = np.abs(plan["r"][:-1] - df) / plan["r"][:-1]

    ax.plot(
        plan["f"][:-1],
        err_step,
        label=f"{props['name']}: r=Δf",
        color=props["color"],
        ls=":",  # Use a consistent dotted style for this constraint type
        lw=2.5,
    )  # Make it thicker to stand out

# --- Final plot setup ---
ax.set_title("Verification of Scheduler Internal Constraints")
ax.set_xlabel("Frequency [Hz]", fontsize=FIG_DEFAULTS["fontsize"])
ax.set_ylabel("Relative Error", fontsize=FIG_DEFAULTS["fontsize"])
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_ylim(1e-20, 0)
ax.grid(True, which="both", linestyle="--", linewidth=0.5)
ax.legend(
    loc="best",
    edgecolor="black",
    fancybox=True,
    shadow=True,
    framealpha=1,
    fontsize=FIG_DEFAULTS["fontsize"] - 1,
)
fig.tight_layout()
plt.show()

In [ ]:
plan_ltf = plans["ltf"]
r0 = plan_ltf["r"]
L0 = plan_ltf["L"]
f0 = plan_ltf["f"]
df0 = np.diff(f0)
i0 = np.arange(plan_ltf["nf"])

plan_new = plans["new_ltf"]
r1 = plan_new["r"]
L1 = plan_new["L"]
f1 = plan_new["f"]
df1 = np.diff(f1)
i1 = np.arange(plan_new["nf"])

fig, ax = plt.subplots(figsize=FIG_DEFAULTS["figsize"], dpi=FIG_DEFAULTS["dpi"])

err_dft_new = np.abs(r1 - (fs / L1)) / (fs / L1)
ax.plot(
    i1,
    err_dft_new * 1e14,
    lw=FIG_DEFAULTS["linewidth"],
    c="tomato",
    ls="-",
    label="r[j] = fs/L[j], New LTF",
)

err_dft_ltf = np.abs(r0 - (fs / L0)) / (fs / L0)
ax.plot(
    i0,
    err_dft_ltf * 1e14,
    lw=FIG_DEFAULTS["linewidth"],
    c="blue",
    ls="--",
    label="r[j] = fs/L[j], LTF",
)


err_step_ltf = np.abs(r0[:-1] - df0) / df0
ax.plot(
    i0[:-1],
    err_step_ltf * 1e14,
    lw=FIG_DEFAULTS["linewidth"],
    c="cyan",
    ls="--",
    label="r[j] ≈ Δf, LTF",
)

err_step_new_ltf = np.abs(r1[:-1] - df1) / df1
ax.plot(
    i1[:-1],
    err_step_new_ltf * 1e14,
    lw=FIG_DEFAULTS["linewidth"],
    c="black",
    label="r[j] ≈ Δf, New LTF (min error)",
)

ax.set_xlabel("Frequency Index, j", fontsize=FIG_DEFAULTS["fontsize"])
ax.set_ylabel("Relative Error * 1e14", fontsize=FIG_DEFAULTS["fontsize"])
ax.tick_params(labelsize=FIG_DEFAULTS["fontsize"])
ax.grid(True)
ax.legend(
    loc="best",
    edgecolor="black",
    fancybox=True,
    shadow=True,
    framealpha=1,
    fontsize=FIG_DEFAULTS["fontsize"] - 1,
    handlelength=2.5,
)
fig.tight_layout()
plt.show()